# Simulateur d'espace with master plan remove

#### But : Identifier les equipements du masterplan présent sur la dernière iteration de IT_Equipment afin de simuler l'espace disponible lorsque tous les equipements seront retirer

Ce workflow a pour but de simuler l'espace disponible apres avoir retirer les equipements qui figure dans le masterplan sur chacun des racks afin de permettre une projection des équipes dans le déplacement du matériel.

DROP table if EXISTS Master_Plan_Global_P2_only ;# MySQL returned an empty result set (i.e. zero rows).
CREATE table Master_Plan_Global_P2_only
as
select * from `Master_Plan_Global`
where room = 'P2'

In [ ]:
DROP VIEW IF EXISTS IT_Equipment_x_Master_plan_P2 ;# MySQL returned an empty result set (i.e. zero rows).
create view IT_Equipment_x_Master_plan_P2
as
select it.ASSET_ID, it.nom, it.SHORT_NAME, it.Modele, it.Fabricant, it.Nom_de_la_salle, it.Nom_de_baie, it.Reperage_ID, it.Hauteur_U, it.Type, mp.SHORT_NAME Master_Plan_name,
  (CASE 
   WHEN mp.short_name is not null and mp.asset_id is null then 'short_name found'
   WHEN mp.short_name is null and mp.asset_id is not null then 'asset_id found'
   WHEN mp.short_name is not null and mp.asset_id is not null then 'found'
   WHEN mp.short_name is null and mp.asset_id is null then 'not found'
   END) Master_plan_status ,
mp.PR_SCOPE, mp.ROOM Master_plan_salle, mp.dalle Master_plan_dalle, mp.RACK_ID Master_plan_rack, mp.Size_U Master_plan_hauteur

 
from `IT_Equipment_v2` it
LEFT JOIN `Master_Plan_Global_P2_only` mp ON it.ASSET_ID = mp.asset_id or it.short_name = mp.short_name
where nom_de_la_salle = 'P2'
GROUP by it.ASSET_ID# MySQL returned an empty result set (i.e. zero rows).

In [ ]:
drop view if exists IT_Equipment_with_equipment_removed ;
create view IT_Equipment_with_equipment_removed
as
select it.* from `IT_Equipment_v2` it 
INNER JOIN `IT_Equipment_x_Master_plan_P2` mp  ON it.ASSET_ID = mp.asset_id or it.short_name = mp.short_name
where master_plan_status = 'not found' and it.Nom_de_la_salle = 'P2'

`rack_capacity_map` : cartographie les racks de toutes les salles afin de rajouter la capacité maximale d'un rack défini à 45 unités U  

In [ ]:
drop view if exists rack_capacity_map ;
create view rack_capacity_map
as 
SELECT it.Nom_de_la_salle, it.Nom_de_baie, '45' as rack_capacity
from `IT_Equipment_with_equipment_removed` it
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie ;

`indicator_it_equipment_simulateur_espace` : indicateur de l'espace disponible pour tous lers rack par rapport à la somme des hauteurs de chaque equipment renseigné sur la CMDB

In [ ]:
drop table if exists indicator_it_equipment_simulateur_espace_raw ;
create table indicator_it_equipment_simulateur_espace_raw
as
SELECT concat(it.Nom_de_la_salle, it.Nom_de_baie) key_id, it.Nom_de_la_salle, it.Nom_de_baie, r.rack_capacity, sum(CEILING(it.Hauteur_U)) slot_occupied, r.rack_capacity - sum(CEILING(it.Hauteur_U)) free_slot,
GROUP_CONCAT(concat(it.slot_u, '/', CEILING(it.Hauteur_U))) list_occupied_slot
from `IT_Equipment_with_equipment_removed` it
INNER JOIN `rack_capacity_map` r ON r.Nom_de_la_salle = it.Nom_de_la_salle and r.Nom_de_baie = it.Nom_de_baie
GROUP BY it.Nom_de_la_salle, it.Nom_de_baie
ORDER by it.Nom_de_la_salle desc , sum(CEILING(it.Hauteur_U)) desc
;